In [ ]:
#Reference: https://github.com/amine0110/nifti2dicom
import pydicom
import nibabel as nib
import time

def set_attributes_save(arr, output_dir, iop, ipp, pixel_spacing, index):

  dicom = pydicom.dcmread("IMG0001.dcm")
  setattr(dicom, "ImageType", ["DERIVED", "SECONDARY"])

  modification_time = time.strftime("%H%M%S")
  modification_date = time.strftime("%Y%m%d")

  setattr(dicom, "SeriesDate", modification_date)
  setattr(dicom, "SeriesTime", modification_time)

  setattr(dicom, "SeriesInstanceUID", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time)
  setattr(dicom, "ImageOrientationPatient", iop) #iop must be list of decimal strings
  setattr(dicom, "ImagePositionPatient", ipp) #ipp must be list of decimal strings

  setattr(dicom, "InstanceCreationDate", time.strftime("%Y%m%d"))
  setattr(dicom, "InstanceCreationTime", time.strftime("%H%M%S"))

  setattr(dicom, "Modality", "CT")
  setattr(dicom, "InstanceNumber", index)
  #setattr(dicom, "SliceThickness", slice_thickness)
  setattr(dicom, "PixelSpacing", pixel_spacing)

  setattr(dicom, "PixelData", arr.tobytes())

  dicom.save_as(output_dir+f"/slice{index}.dcm")

In [ ]:
import nibabel as nib
import numpy as np

def nifti_to_dicom_1file(nifti_dir, output_dir):
  
  nifti_file = nib.load(nifti_dir)
  nifti_affine = nifti_file.affine
  nifti_array = nifti_file.get_fdata()

  direction = nifti_affine[:3, :2]

  direction

  row_cosines = direction[:, 0]
  column_cosines = direction[:, 1]

  magnitude1 = np.linalg.norm(row_cosines)
  magnitude2 = np.linalg.norm(column_cosines)

  normalized_row = row_cosines / magnitude1
  normalized_col = column_cosines / magnitude2

  iop = np.concatenate((normalized_row, normalized_col), axis = 0)

  iop[0] *= float(-1) #RAS -> LPS orientation
  iop[4] *= float(-1)

  iop = [str((float(x))) for x in iop]

  pixelspacing = list(nifti_file.header["pixdim"][1:3])
  pixelspacing = [str(float(x)) for x in pixelspacing]

  num_slices = nifti_array.shape[2]

  for i in range(num_slices):
    slice_ = nifti_array[:, :, i].astype("uint16").transpose()
    #transposing to go from row-major order (which np array uses)
    #to column-major order (dicom standard)
    ipp = nib.affines.apply_affine(nifti_affine, (0, 0, i))
    ipp[0] *= float(-1) 
    ipp[1] *= float(-1) 
    ipp = ["{:.8f}".format((float(x))) for x in ipp]
    
    set_attributes_save(arr = slice_, output_dir=output_dir, iop = iop, ipp = ipp, pixel_spacing = pixelspacing, index = i)